In [2]:
from app import *
%matplotlib inline

app = JupyterSMV(in_features="sites/Sites_lf_geo.json")
app.ui

# day 3 tutorial

```
pip install ipywidgets
pip install ipyleaflet
jupyter nbextension enable --py widgetsnbextension
```
**notes:**
* old code that Yaxing might still want to use is at bottom


In [ ]:
import os                            # core
import json
from io import StringIO              # for python2: import StringIO 

import requests                      # to download SMV data
import numpy as np                   #
import pandas as pd                  #
import xarray as xr                  #
from shapely.geometry import shape   #

import ipywidgets as wg              # widgets and plotting
import ipyleaflet as mwg 
import matplotlib.pyplot as plt
from matplotlib import cm, colors
from IPython.display import display

auth = dict(ORNL_DAAC_USER_NUM=str(32863))             # Jack
url = "https://daac.ornl.gov/cgi-bin/viz/download.pl?" # SMV
hstyle = {"color": "white", "fillOpacity": 0.6}

## SMV Datasets


[*docs/smvdatasets.csv*](docs/smvdatasets.csv) is a copy of the datasets table from the [SMV User Guide](https://daac.ornl.gov/soilmoisture/guide.html). Read it into a `pandas` data frame and display it:

In [ ]:
smvds = pd.read_csv("docs/smvdatasets.csv", index_col="dataset", header=0)
smvds

Example from file:

In [ ]:
df = pd.read_csv("docs/daily-smap-ORNL-DAAC-PccIuo.txt", header=4, index_col="time")
df.index = pd.to_datetime(df.index)        

df.head(5)

An example dataset:

In [ ]:
data = df["AirMOSS_L4_rootzone"].str.split(";", n=2, expand=True)       # split pd column to 3
data = data.replace('', np.nan)                                         # set '' to nan
data = data.astype(float)                                               # set all to float
data.columns = ["AirMOSS_L4_rootzone_"+s for s in ["mean","min","max"]] # set column names

data.head(5)

plot it:

In [ ]:
plt.rcParams['figure.figsize'] = [14, 5]

data["AirMOSS_L4_rootzone_mean"].plot()
data["AirMOSS_L4_rootzone_min"].plot()
data["AirMOSS_L4_rootzone_max"].plot()

---------------------------------------------
## Read USFS data from GeoJSON

The original dataset was a shapefile, but we reprojected and saved as GeoJSON using *ogr2ogr* from the GDAL/OGR binaries package available at OSGeo.

Let's open the GeoJSON and reorganize it as a pandas data frame. Read to a dictionary with `json.load` and print the first feature:

In [ ]:
with open("sites/Sites_lf_geo.json", "r") as f:
    shapes = json.load(f)

features = shapes["features"]

feat = features[0]
feat["properties"]["id"] = 0
feat["properties"]["style"] = {"weight": 1, "fillOpacity": 0.5}

prop = feat["properties"]
prop

A feature's properties (AKA attributes) are stored in the "properties" element of the GeoJSON object:

In [ ]:
geom = feat["geometry"]         # each feature has a geom
geom

Get mean, std from shapefile:

In [ ]:
stats = pd.DataFrame({
    "mean": [v for k,v in prop.items() if "MEAN" in k],
    "std": [v for k,v in prop.items() if "STD" in k]})

stats.head(5)

Use shapely shape to see if inside:

In [ ]:
sgeom = shape(geom)     # Shapely.geometry.shape
bnds = sgeom.bounds
cent = sgeom.centroid

sgeom

Leaflet poly:

In [ ]:
bmap = mwg.basemap_to_tiles(mwg.basemaps.Esri.WorldImagery)
poly = mwg.GeoJSON(data=feat)
points = mwg.LayerGroup()

m1 = mwg.Map(
    layers=(bmap, poly, points,), 
    center=(cent.y, cent.x), 
    zoom=9)

m1

# EASE Grid

Spatial queries to the Soil Moisture Visualizer return data corresponding to 9- by 9-km cells within the EASE grid system. Read about the EASE grid at the NSIDC's web page: https://nsidc.org/data/ease

The next two cells show how to select arrays of EASE grid sample points that fall within an input polygon so that they can be used to submit a series of data requests to the SMV. We will combine everything into one more function (**get_ease**) to use later in our batch processing routine.
      
**Two binary files contain the arrays corresponding to global EASE grid centroid latitudes and longitudes, respectively. Open the two files and read into `numpy` arrays:**

In [ ]:
lats = np.fromfile("docs/EASE2_M09km.lats.3856x1624x1.double", dtype=np.float64).flatten() 
lons = np.fromfile("docs/EASE2_M09km.lons.3856x1624x1.double", dtype=np.float64).flatten()
crds = np.dstack((lats,lons))[0]
crds

Select a 2-dimensional array of EASE grid centroids using some arbitrary latitude, longitude bounds:

In [ ]:
bnds = sgeom.bounds 
bnds

Get the points inside the polygon:

In [ ]:
ease = crds[(bnds[1]<lats)&(lats<bnds[3])&(bnds[0]<lons)&(lons<bnds[2])]
ease

Make some points:

In [ ]:
for p in ease:
    pt = mwg.CircleMarker(                       # map point
        location=(p[0],p[1]),                    # lat,lon tuple
        radius=7,                                # in pixels
        stroke=False,
        fill_opacity=0.6,
        fill_color="black")
    points.add_layer(pt)
    
m1

In [ ]:
points.clear_layers()
point = lambda p: shape({"coordinates": p, "type": "Point"})

for p in ease:
    spt = point([p[1], p[0]])
    if sgeom.contains(spt):
        pt = mwg.CircleMarker(
            location=(p[0],p[1]),
            radius=7,
            stroke=False,
            fill_opacity=0.6,
            fill_color="black")
        points.add_layer(pt)

Make a function that includes all of the logic for getting the list of EASE coordinates inside a polygon:

In [ ]:
def get_ease(geom):
    """ """
     
    bnds = geom.bounds 
    ease = crds[(bnds[1]<lats)&(lats<bnds[3])&(bnds[0]<lons)&(lons<bnds[2])]
    
    pt = lambda p: shape({"coordinates": p, "type": "Point"})
    inpoly = [[p[0],p[1]] for p in ease if geom.contains(pt([p[1], p[0]]))]
    
    return(inpoly)

# Download a SMV dataset with `requests`
Each request to SMV takes a latitude `&lt` and longitude `&ln`. This request is for (30,-100):       
https://daac.ornl.gov/cgi-bin/viz/download.pl?lt=30&ln=-100&d=smap

In [ ]:
lt, ln = ease[0]
url = "https://daac.ornl.gov/cgi-bin/viz/download.pl?lt={lt}&ln={ln}&d=smap".format(lt=lt,ln=ln)
r = requests.get(url, cookies=dict(ORNL_DAAC_USER_NUM="10"))
f = StringIO(r.text)

print("\n".join(f.readlines()[0:10]))

The two functions **txt_to_pd** and **split_pd** do everything we've learned to this point: convert the request response to a text object, then a data frame; and parse the columns of strings into three new columns.

In [ ]:
def txt_to_pd(response_text):
    """Parses response.text to data frame with date index."""
    
    f = StringIO(response_text)                      # get file from string
    df = pd.read_csv(f, header=4, index_col="time")  # read to df
    df.index = pd.to_datetime(df.index)              # convert index to dates
    
    return(df)


def split_pd(col):
    """Splits pd column by ; and set all values to float, nan."""
    
    df = col.str.split(";",n=2,expand=True)           # split col by ;
    df = df.replace('', np.nan)                       # set '' to nan
    df = df.astype(float)                             # set all to float
    df.columns = ["mean","min","max"]                 # add column names
    
    return(df)

We use these repeatedly to request an process the entire grid:

In [ ]:
df = txt_to_pd(r.text)                                # parse response.text to df
dfs = {col: split_pd(df[col]) for col in df.columns}  # loop over cols and split to dfs

dfs["SMAP_rootzone"].tail(5)

## Reformat SMV data as a netCDF-like `xarray.Dataset`
The function below converts SMV outputs to an `xarray.Dataset`. The structure provided by `xarray` is based on pandas, but is better suited (in my opinion) for organizing data that has a spatial component. 

In [ ]:
latatts = dict(
    standard_name="latitude",
    long_name="sample latitude",
    units="degrees_north")

lonatts = dict(
    standard_name="latitude",
    long_name="sample latitude",
    units="degrees_north")

s = xr.DataArray(data=[1], dims=["sample"])
latarr = xr.DataArray(data=[lt], coords=[s], dims=["sample"], attrs=latatts)
lonarr = xr.DataArray(data=[ln], coords=[s], dims=["sample"], attrs=lonatts)

latarr

Now add one more step to the response -> pandas -> split pandas workflow by making an xarray dataset. Print the SMAP_rootzone dataset:

In [ ]:
def pd_to_xr(dataset, df):
    """Makes an xr.Dataset from a pandas column (series) and coords."""
    
    a = smvds.loc[dataset].to_dict()
    x = xr.DataArray(df, name=dataset, attrs=a)
    x = x.rename(dict(dim_1="stat"))
    x.attrs["allnan"] = int(np.isnan(np.nanmean(x.data)))
    
    return(x)


ds = {c: pd_to_xr(c,d) for c,d in dfs.items()}
xds = xr.merge(ds.values())
xds = xds.assign_coords(lat=latarr, lon=lonarr)
xds

And this is what a single SMV dataset looks like:

In [ ]:
xds["SMAP_surface"]

In some cases it may be advantageous to reorder the dimensions over which the data are arranged. You can transpose the 2-d array with [`xarray.Dataset.transpose`](http://xarray.pydata.org/en/stable/generated/xarray.Dataset.transpose.html):

```
xdsT = xds.transpose()
```

### Get the "plottable" datasets
Remember that we added an attribute to each SMV dataset that indicates whether or not the mean\*min\*max array is entirely nodata: *allnan*

Exclude SMV datasets that are entirely nodata using [`xarray.Dataset.filter_by_attrs`](http://xarray.pydata.org/en/stable/generated/xarray.Dataset.filter_by_attrs.html#xarray.Dataset.filter_by_attrs):

In [ ]:
pds = xds.filter_by_attrs(allnan=0)
pds

We can also filter by any of the other attribute(s) that we assigned from the SMV datasets table:

In [ ]:
pds.filter_by_attrs(source="SMAP", soil_zone="rootzone")

### Slice/filter using dimension-based criteria
Filter by the *stat* dimension:

In [ ]:
pds.sel(stat="mean")

or the *time* dimension:

In [ ]:
time = pds.time.data
print(time[10]); print(time[20])

pds.sel(time=slice(time[10],time[20]))

This feature becomes more useful as you add more dimensions to your dataset. We'll use it to filter across three dimensions once we add more sample locations to this dataset.

In [ ]:
pds["SMAP_surface"]

## Make a simple interactive plotting UI
We use the logic above to drive the plotting UI. We can get a list of the attributes to filter by using list comprehension:

In [ ]:
source = list(set([pds[d].attrs["source"] for d in pds]))
stype = list(set([pds[d].attrs["type"] for d in pds]))
soil_zone = list(set([pds[d].attrs["soil_zone"] for d in pds]))

print(source); print(stype); print(soil_zone)

GRACE has very few observations, so we don't really need the source filter. And all data are from spaceborne datasets, so the only relevant attribute filter for this dataset is the *soil_zone*:

In [ ]:
dates = pds.sel(stat="mean").dropna(dim="time", how="all").time.data
dates = dates.astype('M8[D]')

time_slider = wg.SelectionRangeSlider(
    options=dates, 
    index=(0, len(dates)-1),
    continuous_update=False,
    layout=wg.Layout(width="auto"))

widgets = dict(
    Time=time_slider, 
    By=["None", "year", "month", "week", "day"],
    Zone=['surface', 'rootzone'],
    Mean=True, Min=True, Max=True)

# needs to run twice to switch from inline -->
%matplotlib notebook

Build and display the plot ui:

In [ ]:
%matplotlib notebook
plt.rcParams['figure.figsize'] = [12, 5]

fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)


def update(Time, By, Zone, Mean, Min, Max):
    """ """

    stat = [True]*3 if not any([Mean,Min,Max]) else [Mean,Min,Max]
    data = pds.sel(stat=stat)                      # filter by stats
    
    data = data.filter_by_attrs(soil_zone=Zone)    # filter by attributes

    data = data.sel(time=slice(Time[0],Time[1]))   # filter by time;
    
    xaxis = "time" if By == "None" else By         # new plot interval
    if By is not "None":
        data = data.sel(stat="mean")
        data = data.groupby("time."+str(By)).mean()

    ax.clear()                                     # clear plot
    for d in data:                                 # loop over vars
        data[d].plot.line(x=xaxis, ax=ax)          # add line
    fig.canvas.draw()                              # draw 


p = wg.interactive(update, **widgets);
display(p)

## Organize a series of SMV samples into similar structure

The capabilities of xarray aren't obvious until you add a second dimension to the data (excluding the unnecessary *stats* dim). You can do everything we just did with pandas. Let's look at our USFS polygon again with a new map:

In [ ]:
points = mwg.LayerGroup()
polys = mwg.LayerGroup(layers=(poly,))
m2 = mwg.Map(layers=(polys, points, bmap), center=(cent.y, cent.x), zoom=9)

m2

### Container for point samples
`Sample(<id>,<lat>,<lon>)`
* id: an integer id unique to the sample within it's input polygon
* lat, lon: latitude, longitude numerics

In [ ]:
url = "https://daac.ornl.gov/cgi-bin/viz/download.pl?"

class Sample(object):

    def __init__(self, i, lat, lon):
        """Inits with id,lat,lon; makes request string, map point."""
        self.id, self.lat, self.lon = i, lat, lon               # id, lat, lon
        self.rurl = url+"lt={0}&ln={1}&d=smap".format(lt,ln)    # request url     
        self.pt = mwg.CircleMarker(                             # map point
            location=(lat,lon),                                 # lat,lon tuple
            radius=7,                                           # in pixels
            stroke=False,
            fill_opacity=0.6,
            fill_color="black")

    def update(self, **kwargs):
        for arg, val in kwargs.items():
            setattr(self.pt, arg, val)
        
    def submit(self):
        """Called by parent. Downloads url. Updates status."""
        self.response = requests.get(self.rurl, cookies=auth)   # submit SMV request
        self.df = txt_to_pd(self.response.text)                 # read to pandas df

Use the function we made before `get_ease` to get a list of EASE points inside the polygon, make a Sample for each, and organize inside a data frame. Print the first five rows of the data frame and display the updated map:

In [ ]:
samples = []
for i, pt in enumerate(get_ease(sgeom)):
    s = Sample(i, pt[0], pt[1])                # make a Sample instance
    points.add_layer(s.pt)                     # add map pt to points group
    samples.append((i, pt[0], pt[1], s, None)) # append tuple to the list

samples = pd.DataFrame(                        # convert list of tuples to df
    samples, 
    columns=["id", "lat", "lon", "samp", "xr"])

print(samples.head(5)); m2                     # display

Add a couple more widgets purely for aesthetics:

In [ ]:
n = len(samples.samp)
progress = wg.IntProgress(value=0, min=0, max=n, description="Progress: ", layout=wg.Layout(width="95%"))

def submit_handler(b):
    submit.disabled = True               # disable submit button
    for samp in samples.samp:            # loop over sample pts
        progress.value += 1              # update progress bar
        samp.update(                     # update point style
            stroke=True, 
            color="white", 
            opacity=0.6)
        samp.submit()                    # download the data

submit = wg.Button(description='Submit', button_style='success')
submit.on_click(submit_handler)

wg.VBox([m2,wg.HBox([submit,progress])])

Hopefully you didn't have any trouble downloading the data. Remember we made the class that binds a map marker to several other items including a pandas data frame that gets created when the sample is retrieved from the SMV.

Check the data frame for sample zero:

In [ ]:
samples0 = samples.iloc[0]
print(samples0); samples0.samp.df.tail(5)

This looks familiar. Use the steps that we learned before to convert to an xarray dataset:

In [ ]:
s0 = xr.DataArray(data=[samples0.id], dims=["sample"])
y0 = xr.DataArray(data=[samples0.lat], coords=[s0], dims=["sample"], attrs=latatts)
x0 = xr.DataArray(data=[samples0.lon], coords=[s0], dims=["sample"], attrs=lonatts)

df0 = samples0.samp.df                                  # get the sample df
dfs0 = {col: split_pd(df0[col]) for col in df0.columns} # loop over cols and split to dfs
ds0 = {c: pd_to_xr(c,d) for c,d in dfs0.items()}        # make xr datasets for each smv
xds0 = xr.merge(ds0.values())                           # merge to one xr dataset
xds0 = xds0.assign_coords(lat=y0, lon=x0)               # add coordinate arrays sample 
xds0

While we're at it, wrap all of that up in a function to apply to all of the samples:

In [ ]:
def get_sample_xr(samp):
    """ """
    
    # get sample, lat, lon xr arrays
    s = xr.DataArray(data=[samp.id], dims=["sample"])
    y = xr.DataArray(data=[samp.lat], coords=[s], dims=["sample"], attrs=latatts)
    x = xr.DataArray(data=[samp.lon], coords=[s], dims=["sample"], attrs=lonatts)

    df = samp.df                                         # get the sample df
    dfs = {col: split_pd(df[col]) for col in df.columns} # loop over cols and split to dfs
    ds = {c: pd_to_xr(c,d) for c,d in dfs.items()}       # make xr datasets for each smv
    xds = xr.merge(ds.values())                          # merge to one xr dataset
    xds = xds.assign_coords(lat=y, lon=x)                # add coordinate arrays
    
    return(xds)
    

for ix, row in samples.iterrows():                       # loop over samples df
    samples.at[ix, "xr"] = get_sample_xr(row.samp)       # add xr dataset to col
    
samples

Check sample ten:

In [ ]:
samples.iloc[10].xr

Hopefully, if we organized the data correctly, we can concatenate along the sample dimension:

In [ ]:
xds0 = xr.concat(samples.xr.tolist(), "sample")
xds0

Save as a netCDF. The dataset needs another variable and some special attributes to comply with CF (explain CF):
* a variable that describes the sequence that makes up the sample dimension
* a dataset-level attribute that inidcates the dataset's *featureType*

In [ ]:
# loop over sample dim sequence and make strings like: "sample##"
sample_name_data = ["sample"+("%02d" % s) for s in xds0.sample.data]

xds0["sample_name"] = xr.DataArray(        # make an xr array
    data=sample_name_data, 
    dims=["sample"], 
    attrs=dict(                            # cf attributes
        long_name="sample name", 
        cf_role="timeseries_id"))

xds0.attrs.update({
    "convention": "CF-1.6", 
    "featureType": "timeSeries",
    "source": "Soil Moisture Visualizer",
    "institution": "Oak Ridge National Laboratory Distributed Active Archive Center"})

xds0

Save with [`xarray.Dataset.to_netcdf`](http://xarray.pydata.org/en/stable/generated/xarray.Dataset.to_netcdf.html#xarray.Dataset.to_netcdf):

In [ ]:
xds0.to_netcdf("23samples.nc")

## Extend the plotting UI
We probably have more options for filters with a dataset this size. Let's see:

In [ ]:
xpds = xds0.filter_by_attrs(allnan=0)
xpds

## better colors
1. use [`numpy.linspace`](https://docs.scipy.org/doc/numpy/reference/generated/numpy.linspace.html) to make an array of evenly-spaced values between 0-1 
2. map values to **Set3** in [`matplotlib.cm`](https://matplotlib.org/api/cm_api.html) | [colormap reference](https://matplotlib.org/gallery/color/colormap_reference.html)
3. convert to hexadecimal with [`matplotlib.colors.to_hex`](https://matplotlib.org/api/_as_gen/matplotlib.colors.to_hex.html#matplotlib.colors.to_hex)

In [ ]:
cspace = np.linspace(0.0, 1.0, len(features)) # 1
rgb = cm.Set3(cspace)                         # 2
cols = [colors.to_hex(c[0:3]) for c in rgb]   # 3

cols

MAybe give them the option to import shapefile?
```
with open("sites/Sites_lf_geo.json", "r") as f:
    shapes = json.load(f)
features = shapes["features"]
```


In [ ]:
m1,m2,s0,df0,dfs0,xds0,samples = None,None,None,None,None,None,None
# -----------------------------------------------------------------

site_details = """
{FORESTNAME} ({FORESTNUMB})
{DISTRICTNA} ({DISTRICTNU})
REGION:   {REGION}
ACRES:    {GIS_ACRES}
MIN:      {MIN}
MEDIAN:   {MEDIAN}
MAX:      {MAX}
RANGE:    {RANGE}
SUM:      {SUM}
VARIETY:  {VARIETY}
MINORITY: {MINORITY}
MAJORITY: {MAJORITY}
COUNT:    {COUNT}
"""

out_style = dict(width="30%", height="400px", overflow_y="scroll", overflow_x="hidden", border="1px solid gray")
out = Output(layout=Layout(**out_style))

out.clear_output()
with out:
    print(site_details.format(**lo.site))

In [ ]:
import ipywidgets as wg
import ipyleaflet as mwg
from ipywidgets import Layout, Button, IntProgress, Output, HBox, VBox, HTML
from ipyleaflet import Map, LayerGroup, GeoJSON, CircleMarker

basemap = mwg.basemap_to_tiles(mwg.basemaps.Esri.WorldImagery)
polys = LayerGroup()
points = LayerGroup()

map_center = (32.75, -109)
mapw = Map(
    layers=(basemap, polys, points,), 
    center=map_center, 
    zoom=7, 
    scroll_wheel_zoom=True)

submit = Button( 
    description='Submit', 
    disabled=True, 
    button_style='success')

progress = IntProgress(
    description="Progress: ", 
    layout=Layout(width="95%"))

ui = VBox([mapw, HBox([submit, progress])])

Sample class, few small changes:

In [ ]:
# ----------------------------------------------------------------------------
url = "https://daac.ornl.gov/cgi-bin/viz/download.pl?"

pt_style = dict(radius=7, fill_opacity=0.6, fill_color="black", stroke=False)
pt_status_on = dict(stroke=True, color="white", opacity=0.6)
pt_status_off = dict(stroke=False, color="black", opacity=0.6)


class Sample(object):

    def __init__(self, i, lat, lon):
        """Inits with id,lat,lon; makes request string, map point."""
        self.id, self.lat, self.lon = i, lat, lon
        self.rurl = url+"lt={0}&ln={1}&d=smap".format(lat,lon)  # request url
        self.pt = CircleMarker(location=(lat, lon), **pt_style) # map point
        self.on = False                                         # on/off status

    def update(self, **kwargs):
        for arg, val in kwargs.items():
            setattr(self.pt, arg, val)
    
    def toggle(self, event, type, coordinates):
        opac = 0.1 if self.on else 0.6
        self.update(opacity=opac)
        self.on = False if self.on else True
        
    def submit(self):
        """Called by parent. Downloads url. Updates status."""
        self.response = requests.get(self.rurl, cookies=auth)   # submit SMV request
        self.df = txt_to_pd(self.response.text)                 # read to pandas df
        self.xr = get_sample_xr(self)                           # get xarray dataset
        self.pt.on_click(self.toggle)                           # callback on click
        self.on = True                                          # toggle on

# ----------------------------------------------------------------------------
lyr_style = lambda c: {"color": c, "fillColor": c, "weight": 1, "fillOpacity": 0.4}
lyr_hstyle = {"color": "white", "fillOpacity": 0.8}
statcheck = lambda k: k.split("_")[0] not in ["MEAN","STD","Count", "style"]


def mgeo(i, feat, col):
    feat["properties"].update({
        "id": i, 
        "style": lyr_style(col)})
    return(dict(data=feat, hover_style=lyr_hstyle))


class Layer(object):

    def __init__(self, i, feat, col=None):
        """Inits with id,lat,lon; makes request string, map point."""
        self.id = i
        self.feat = feat
        
        self.sgeom = shape(feat["geometry"])
        self.ease = get_ease(self.sgeom)                        # get ease points
        self.cent = self.sgeom.centroid                         # get centroid
        self.lat, self.lon = self.cent.y, self.cent.x           # get lat, lon
        
        prop = feat["properties"]
        mean = [v for k,v in prop.items() if "MEAN" in k]
        std = [v for k,v in prop.items() if "STD" in k]
        self.stats = pd.DataFrame({"mean": mean, "std": std})
        self.site = {k:v for k,v in prop.items() if statcheck(k)}
        
        lyr = mgeo(i, feat, col)
        self.layer = GeoJSON(**lyr)
        self.layer.on_click(self.toggle)

        self.on, self.dl = False, False                    # on/off, dl status
        
    def update(self, **kwargs):
        for arg, val in kwargs.items():
            setattr(self.layer, arg, val)
    
    def toggle(self, **kwargs):
        """Routine for when a new USFS polygon is selected."""
        if list(kwargs.keys()) != ['event', 'properties']: # check event
            return(None)                                   # skip basemap
        self.on = False if self.on else True               # update status

In [ ]:
prog = lambda m: dict(min=0, max=m, value=0)
xrds = lambda l: xr.concat([s.xr for s in l], "sample")


def get_on_lyrs(column=None):
    """
    Returns a subset of layers df, only "on" layers. If keyword 
    argument 'column' will return only that column.
    """
    on = [i for i,row in layers.iterrows() if row["layer"].on]
    sdf = layers.iloc[on][column] if column else layers.iloc[on]
    return(sdf)


def submit_handler(b):
    """Resets UI and sends requests to SMV when new submit."""
    
    lyron = get_on_lyrs()
    for i, row in lyron.iterrows():             # loop over samples col
        if not row["layer"].dl:                 # if not downloaded yet
            samp = row.samples["samp"].tolist() # get samples
            for a,v in prog(len(samp)).items(): # reset progress bar
                setattr(progress, a, v)
            for s in samp:                      # loop over sample pts
                progress.value += 1             # update progress bar
                s.update(**pt_status_on)        # update style
                s.submit()                      # download the data
            layers.at[i,"xr"] = xrds(samp)      # make xr dataset
            row["layer"].dl = True              # set dl status to True
    submit.disabled = True                      # disable submit button

submit.on_click(submit_handler)

# ----------------------------------------------------------------------------

def layer_click_handler(**kwargs): 
    """
    Routine for when a new USFS polygon is selected. Layer.toggle
    internal updater should evaluate first.
    """
    if list(kwargs.keys()) != ['event', 'properties']: # check event
         return(None)                         # skip basemap
    i = int(kwargs["properties"]["id"])       # set selected poly id
    l = layers.iloc[i]                        # get row for selected
    lo = l.layer                              # get Layer class inst
    pteval = points.add_layer if lo.on else points.remove_layer
    pteval(l["points"])                       # update layer status;
    submit.disabled = True if len(get_on_lyrs())==0 else False
    if lo.on:
        mapw.center, mapw.zoom = (lo.lat,lo.lon), 9 # ctr,zoom map

### Make all layers and add to map widget

In [ ]:
sample_header = ["id","lat","lon","samp"]
layer_header = ["id","lat","lon","layer","samples","points","xr"]

layers = []                                   # a temporary list 
for i, feat in enumerate(features):           # loops over USFS poly feats
    
    poly = Layer(i, feat, cols[i])            # get Layer class
    poly.layer.on_click(layer_click_handler)  # set global callback
    polys.add_layer(poly.layer)               # add to polys layer group

    pts, samps = LayerGroup(), []             # points layer group; Samples
    for j, p in enumerate(poly.ease):         # loop over EASE grid pts
        s = Sample(j, p[0], p[1])             # make a Sample instance
        pts.add_layer(s.pt)                   # add to points layer group
        samps.append((j, p[0], p[1], s))      # append tuple to the list  
        
    samps = pd.DataFrame(samps, columns=sample_header)       # samples df
    layers.append((i,poly.lat,poly.lon,poly,samps,pts,None)) # append
    
layers = pd.DataFrame(layers, columns=layer_header)          # layers df

In [ ]:
ui